In [1]:
import numpy as np
import random
import numpy.linalg as lia
import pandas as pd
import h5py
import random
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn import datasets
from sklearn.cluster import KMeans

#to plot images
import matplotlib.image as mpimg 
#to read images

import cv2 
#open CV library for Python
import numpy as np
import torch
import torchvision
from torchvision.ops import nms
import torchvision.transforms as transforms
from statistics import stdev

In [2]:
!pip install opencv-python==4.4.0.44
!pip install opencv-contrib-python==4.4.0.44
!pip list

Package                Version
---------------------- -------------------
-pencv-python          4.4.0.46
absl-py                0.11.0
argon2-cffi            20.1.0
astunparse             1.6.3
async-generator        1.10
attrs                  20.3.0
backcall               0.2.0
bleach                 3.2.1
cachetools             4.1.1
certifi                2020.12.5
cffi                   1.14.4
chardet                3.0.4
cloudpickle            1.6.0
colorama               0.4.4
cycler                 0.10.0
cytoolz                0.11.0
dask                   2020.12.0
decorator              4.4.2
defusedxml             0.6.0
entrypoints            0.3
gast                   0.3.3
google-auth            1.23.0
google-auth-oauthlib   0.4.2
google-pasta           0.2.0
grpcio                 1.34.0
h5py                   2.10.0
idna                   2.10
imagecodecs            2020.5.30
imageio                2.9.0
importlib-metadata     2.0.0
imutils                0.5.3
ipykern

In [3]:
def loadBGRImage(path):
    image = BGR(cv.imread(path))
    return image

def loadGreyImage(path):
    image = cv.imread(path, cv.IMREAD_GRAYSCALE)
    return image

def BGR(image):
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image

def displayGreyImage(image, imageName):
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap = 'gray')
    plt.title(imageName)
    plt.show()

def displayGreyWindows(image, imageName):
    plt.figure(figsize=(3, 3))
    plt.imshow(image, cmap = 'gray')
    plt.title(imageName)
    plt.show()
    
def displayBGRImage(image, imageName, size):
    plt.figure(figsize=(size, size))
    plt.imshow(image)
    plt.title(imageName)
    plt.show()
    
def displayBGRImageLarge(image, imageName):
    plt.figure(figsize=(18, 18))
    plt.imshow(image)
    plt.title(imageName)
    plt.show()
    
def imageSideBySide(images, imageNames,size):
    row = np.ceil(len(images)/20)
    fig=plt.figure(figsize=(size, size/2))
    for i, image in enumerate(images):
        fig.add_subplot(row, 20, i+1)
        plt.imshow(image, cmap='gray')
        plt.title(imageNames[i])
    plt.show()

In [4]:
start = 25, 26
size = 14, 12

f = h5py.File('MNIST_synthetic.h5', 'r')

list(f.keys())

['test_dataset', 'train_dataset', 'train_labels']

In [5]:
train_dataset = np.squeeze(np.array(f["train_dataset"])).copy()
train_labels = np.squeeze(np.array(f["train_labels"])).copy()
test_dataset = np.squeeze(np.array(f["test_dataset"])).copy()

singulars_digits = []
singulars_labels = []

doubles_digits = []
doubles_labels = []

triples_digits = []
triples_labels = []

quadruples_digits = []
quadruples_labels = []

quintuples_digits = []
quintuples_labels = []


for i, labels in enumerate(train_labels):
    if labels[1] == 10:
        singulars_digits.append(train_dataset[i])
        singulars_labels.append(train_labels[i])
        
    if labels[1] != 10 and labels[2] == 10:
        doubles_digits.append(train_dataset[i])
        doubles_labels.append(train_labels[i])
        
    if labels[2] != 10 and labels[3] == 10:
        triples_digits.append(train_dataset[i])
        triples_labels.append(train_labels[i])
        
    if labels[3] != 10 and labels[4] == 10:
        quadruples_digits.append(train_dataset[i])
        quadruples_labels.append(train_labels[i])
        
    if labels[4] != 10:
        quintuples_digits.append(train_dataset[i])
        quintuples_labels.append(train_labels[i])

In [6]:
singulars_digits = np.array(singulars_digits)     
doubles_digits = np.array(doubles_digits)  
triples_digits = np.array(triples_digits)    
quadruples_digits = np.array(quadruples_digits)    
quintuples_digits = np.array(quintuples_digits)    

singulars_labels = np.array(singulars_labels).T[0]
doubles_labels = np.array(doubles_labels).T[0:2].T
triples_label = np.array(triples_labels).T[0:3].T
quadruples_label = np.array(quadruples_labels).T[0:4].T
quintuples_label = np.array(quintuples_labels).T[0:5].T

In [7]:
final_singulars_dataset = []
final_singulars_labels = list(singulars_labels)
for i, digit in enumerate(singulars_digits):
    final_singulars_dataset.append(digit[26:38,26:38])
    
final_doubles_dataset = []
final_doubles_labels = []
for i, digit in enumerate(doubles_digits):
    final_doubles_dataset.append(digit[26:38,20:32])
    final_doubles_dataset.append(digit[26:38,32:44])
    final_doubles_labels.append(doubles_labels[i][0])
    final_doubles_labels.append(doubles_labels[i][1])

final_triples_dataset = []
final_triples_labels = []
for i, digit in enumerate(triples_digits):
    final_triples_dataset.append(digit[26:38,14:26])
    final_triples_dataset.append(digit[26:38,26:38])
    final_triples_dataset.append(digit[26:38,38:50])
    final_triples_labels.append(triples_labels[i][0])
    final_triples_labels.append(triples_labels[i][1])
    final_triples_labels.append(triples_labels[i][2])
    
final_quadruples_dataset = []
final_quadruples_labels = []
for i, digit in enumerate(quadruples_digits):
    final_quadruples_dataset.append(digit[26:38,8:20])
    final_quadruples_dataset.append(digit[26:38,20:32])
    final_quadruples_dataset.append(digit[26:38,32:44])
    final_quadruples_dataset.append(digit[26:38,44:56])
    final_quadruples_labels.append(quadruples_labels[i][0])
    final_quadruples_labels.append(quadruples_labels[i][1])    
    final_quadruples_labels.append(quadruples_labels[i][2])
    final_quadruples_labels.append(quadruples_labels[i][3]) 
    
final_quintuples_dataset = []
final_quintuples_labels = []
for i, digit in enumerate(quintuples_digits):
    final_quintuples_dataset.append(digit[26:38,2:14])
    final_quintuples_dataset.append(digit[26:38,14:26])
    final_quintuples_dataset.append(digit[26:38,26:38])
    final_quintuples_dataset.append(digit[26:38,38:50])
    final_quintuples_dataset.append(digit[26:38,50:62])
    final_quintuples_labels.append(quintuples_labels[i][0])
    final_quintuples_labels.append(quintuples_labels[i][1])
    final_quintuples_labels.append(quintuples_labels[i][2])
    final_quintuples_labels.append(quintuples_labels[i][3])
    final_quintuples_labels.append(quintuples_labels[i][4])

In [8]:
merged_dataset = final_singulars_dataset + final_doubles_dataset + final_triples_dataset + final_quadruples_dataset + final_quintuples_dataset
merged_labels = final_singulars_labels + final_doubles_labels + final_triples_labels + final_quadruples_labels + final_quintuples_labels

In [9]:
count = 0

for digit in range(10):
    digit_count = list(merged_labels).count(digit)
    print(f"number of {digit}'s in train dataset: {digit_count}")
    count += digit_count

print("total number of digits in train dataset:", count)

number of 0's in train dataset: 16379
number of 1's in train dataset: 18819
number of 2's in train dataset: 17095
number of 3's in train dataset: 17220
number of 4's in train dataset: 16393
number of 5's in train dataset: 15275
number of 6's in train dataset: 16401
number of 7's in train dataset: 17611
number of 8's in train dataset: 16175
number of 9's in train dataset: 16518
total number of digits in train dataset: 167886


In [10]:
digitsTrainingSetSize = int(np.ceil(0.8 * len(merged_dataset)))
digitsValidationSetSize = int(len(merged_labels) - digitsTrainingSetSize)

xValidationSet = []
yValidationSet = []

for index, digit in enumerate(merged_dataset[0:digitsValidationSetSize]):
    xValidationSet.append(digit)
    yValidationSet.append(merged_labels[index])

xTrainingSet = []
yTrainingSet = []

start = len(xValidationSet)

for i, digit in enumerate(merged_dataset[start:]):
    xTrainingSet.append(digit)
    yTrainingSet.append(merged_labels[i+start])

## Finding all digits in an image

In [11]:
def find_digits(train_dataset, size):
    thresh_train_dataset = train_dataset.copy()
    thresh_train_dataset[thresh_train_dataset>30] = 255
    thresh_train_dataset[thresh_train_dataset!=255] = 0
    #displayGreyImage(thresh_train_dataset,"")
    i,j = np.where(thresh_train_dataset[:,:]!=0)
    #print(i)
    bottom_bound = np.min(i)
    top_bound = np.max(i)
    left_bound = np.min(j)
    right_bound = np.max(j)
    #print(top_bound, bottom_bound)

    h = top_bound-bottom_bound
    w = right_bound-left_bound

    bounded_train_dataset = train_dataset[bottom_bound-1:top_bound+2,left_bound-1:right_bound+2]
    thresh = bounded_train_dataset.copy()
    thresh[thresh>30] = 255
    thresh[thresh!=255] = 0
    #displayGreyImage(bounded_train_dataset,"")

    #ret, thresh = cv2.threshold(bounded_train_dataset, 30, 255, 0)
    #seg = np.where(np.any(thresh, axis=0)==0)
    seg = np.where(np.any(thresh, axis=0)==0)

    #print(seg)
    seg_list = np.asarray(seg)
    seg_list = seg_list[0]
    #print(seg_list)

    from statistics import stdev
    sd =0 # remove this and its return when done
    if len(seg_list)>2:
        # create a list of the gaps between the consecutive values
        gaps = [y - x for x, y in zip(seg_list[:-1], seg_list[1:])]
        # have python calculate the standard deviation for the gaps
        sd = stdev(gaps)
        #print(sd)

        # create a list of lists, put the first value of the source data in the first
        lists = [[seg_list[0]]]
        for x in seg_list[1:]:
            # if the gap from the current item to the previous is more than 1 SD
            # Note: the previous item is the last item in the last list
            # Note: the '> 1' is the part you'd modify to make it stricter or more relaxed
            if sd<1.5 and sd>0:
                if (x - lists[-1][-1]) / (sd+1e-18) > 1.5:
                    # then start a new list
                    lists.append([])
              # add the current item to the last list in the list
                lists[-1].append(x)
            elif sd==0:
                if (x - lists[-1][-1])>1:
                    lists.append([])
                lists[-1].append(x)
            else:
                if (x - lists[-1][-1]) / (sd+1e-18) > 0.8:
                  # then start a new list
                  lists.append([])
                # add the current item to the last list in the list
                lists[-1].append(x)

        splits = np.asarray([np.ceil(np.mean(lists[i])) for i in range(len(lists))]).astype(int)

    else:
        splits = np.asarray(seg_list)
      #print(splits)

    n_digits = len(splits)-1
    #digits = np.zeros(n_digits, )
    digits = []
    for i in range(n_digits):
        temp = bounded_train_dataset[:,splits[i]:splits[i+1]]
        # if temp is less than recommeded size first pad on left and then on both sides
        temp_padded = temp.copy()
        if (temp.shape[0] != size) or  (temp.shape[1] != size):
            diff_y = size-temp.shape[0]
            split_diff_y = diff_y//2
            remainder_diff_y = diff_y%2
            diff_x = size-temp.shape[1]
            split_diff_x = diff_x//2
            remainder_diff_x = diff_x%2
            #print(diff_y)
            #print(split_diff_y)
            temp_padded = np.pad(temp, ((split_diff_y+remainder_diff_y,split_diff_y),(split_diff_x+remainder_diff_x,split_diff_x)))

        digits.append(temp_padded)
      #displayGreyImage(train_dataset[bottom_bound-1:top_bound+2,left_bound-1:right_bound+2],train_labels[367])
    return digits, n_digits

In [12]:
def create_boxes_hacks(image, dig_centers):
    
    n_dig = len(dig_centers)
    digits = []
    if n_dig == 1:
        digits.append(image[26:38,26:38])
    elif n_dig == 2:
        digits.append(image[26:38,20:32])
        digits.append(image[26:38,32:44])
    elif n_dig == 3:
        digits.append(image[26:38,14:26])
        digits.append(image[26:38,26:38])
        digits.append(image[26:38,38:50])
    elif n_dig == 4:
        digits.append(image[26:38,8:20])
        digits.append(image[26:38,20:32])
        digits.append(image[26:38,32:44])
        digits.append(image[26:38,44:56])
    elif n_dig == 5:
        digits.append(image[26:38,2:14])
        digits.append(image[26:38,14:26])
        digits.append(image[26:38,26:38])
        digits.append(image[26:38,38:50])
        digits.append(image[26:38,50:62])

    return digits

In [13]:
import random as rng
rng.seed(12345)
def thresh_callback(src, val=30):
    threshold = val

    src_gray = cv2.blur(src, (1,1))

    #src_gray = bounding_box(src_gray,30)

    canny_output = cv2.Canny(src_gray, threshold, threshold * 2)
    
    
    contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])
    

    arr_centers = np.asarray(centers)
    arr_radius  = np.asarray(radius)

    centers_inds = np.argsort(arr_centers[:,0])
    sorted_centers = arr_centers[centers_inds[::]]
    sorted_radius = arr_radius[centers_inds[::]]
    """
    print("before:", arr_centers)
    print(arr_radius)
    print("after:", sorted_centers)
    print(sorted_radius)
    """

    new_centers = [sorted_centers[i] for i in range(len(sorted_centers)) if sorted_radius[i]>1 and sorted_radius[i]<7]
    new_radius  = [sorted_radius[i] for i in range(len(sorted_radius)) if sorted_radius[i]>1 and sorted_radius[i]<7] 
    
    """
    for i, x in enumerate(new_centers):
      print("new centers", x,"new radius", new_radius[i])
    """

    """
    print(boundRect)
    """

    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

    """
    for i in range(len(centers)):
      color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
      cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)
    """
    
    for i in range(len(new_centers)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv2.circle(drawing, (int(new_centers[i][0]), int(new_centers[i][1])), int(new_radius[i]), color, 2)
    
    """
    displayGreyImage(drawing, "contours")
    """
    
    boxes = []
    scores = []
    boxes_tensor = torch.empty(size=(len(new_centers), 4))
    scores_tensor = torch.empty(len(new_centers))
    for i, x in enumerate(new_centers):
        #print(int(x[0]-new_radius[i]))
        bottom = np.floor(x[1]-new_radius[i]).astype(int)
        top = np.ceil(x[1]+new_radius[i]).astype(int)
        left = np.floor(x[0]-new_radius[i]).astype(int)
        right = np.ceil(x[0]+new_radius[i]).astype(int)
        
        """
        print(f'bottom: {bottom} top: {top} left: {left} right: {right}')
        """

        box = src_gray[bottom:top, left:right]
        score = box.sum()
        scores.append(score)

        #displayGreyImage(box, i)
        boxes.append(box)
        """
        displayGreyImage(boxes[i], i)
        """
        #box_tensor = torch.tensor([bottom, left, top, right])
        #boxes_tensor = torch.cat([box_tensor],0)
        boxes_tensor[i] = torch.tensor([left, bottom, right, top])
        scores_tensor[i] = torch.tensor([score], dtype=torch.float32)

    
    """
    print(f"boxes:{boxes_tensor} scores: {scores_tensor}")
    """

    
    max_boxes_ind = nms(boxes = boxes_tensor, scores = scores_tensor, iou_threshold=0.1)
    
    """
    print(max_boxes_ind)
    """
    
    #cv2_imshow(drawing)
    
    j_list = np.asarray(new_centers)
    """
    print(j_list[:,0])
    """
    j_list = np.asarray(j_list[:,0])
    j_list = np.sort(j_list)
    """
    print("jlist", j_list)
    #print(j_list[1]-j_list[0])
    """

    # create a list of the diffirence in centers between consecutive values
    gaps = [y - x for x, y in zip(j_list[:-1], j_list[1:])]
    """
    print(gaps)
    """
    
    # have python calculate the standard deviation for the gaps
    #sd = stdev(gaps)
    """
    print(sd)
    """
    # create a list of lists, put the first value of the source data in the first
    lists = [[j_list[0]]]
    """
    print(lists)
    """
    radius_lists = [[new_radius[0]]]
    mean_radius = np.median(new_radius)
    #print(mean_radius)
    for i, x in enumerate(j_list[1:]):
        """
        print(x- lists[-1][-1])
        print(new_radius[i])
        
        print(new_radius[i+1])
        print(x-new_radius[i] - (lists[-1][-1]+new_radius[i-1]))
        """
        # if the gap from the current item to the previous is more than 1 SD
        # Note: the previous item is the last item in the last list
        # Note: the '> 1' is the part you'd modify to make it stricter or more relaxed
        if np.abs(x - (lists[-1][-1])) > 6: #radius[i] (new_radius[i]+new_radius[i+1])*2/3
            lists.append([])
            radius_lists.append([])
        lists[-1].append(x)
        radius_lists[-1].append(new_radius[i])
    """
    print(lists)
    print(radius_lists)
    """
    digit_centers = np.asarray([np.mean(lists[i]) for i in range(len(lists))]).astype(int)
    digit_radius = np.asarray([np.max(radius_lists[i]) for i in range(len(radius_lists))]).astype(int)

    final_boxes = create_boxes_hacks(src, digit_centers)

    """
    for i in range(len(final_boxes)):
        displayGreyImage(final_boxes[i],"")
    """
    
    n_digits = len(final_boxes)

    """
    print(digit_centers, digit_radius)
    print(digit_centers)
    print(len(digit_centers))
    """
    return final_boxes, n_digits

In [14]:
wrong = 0 
wrong_arr = []
wrong_arr_index = []
correct = []

for i, sample in enumerate(train_dataset):
    snips, n_dig = thresh_callback(sample)
    real_num_digits = 5 - list(train_labels[i]).count(10)

    if(n_dig != real_num_digits):
        print("incorrect at index:", i)
        wrong += 1
        wrong_arr.append(snips)
        wrong_arr_index.append(i)  

print("total percentage incorrect:", wrong / len(train_dataset) * 100, "%")

incorrect at index: 2903
incorrect at index: 4555
incorrect at index: 7230
incorrect at index: 7472
incorrect at index: 12016
incorrect at index: 12127
incorrect at index: 13485
incorrect at index: 16017
incorrect at index: 18127
incorrect at index: 19418
incorrect at index: 19580
incorrect at index: 19752
incorrect at index: 20317
incorrect at index: 20829
incorrect at index: 24262
incorrect at index: 25284
incorrect at index: 28261
incorrect at index: 32014
incorrect at index: 32944
incorrect at index: 32954
incorrect at index: 33012
incorrect at index: 38206
incorrect at index: 48432
incorrect at index: 50390
incorrect at index: 51039
total percentage incorrect: 0.044642857142857144 %


In [15]:
num_test_digits = 0

for i, sample in enumerate(test_dataset):
    images, n_digits = thresh_callback(sample)
   
    num_test_digits += n_digits

divs = []

for div in range(1, 100):
    if(num_test_digits % (div) == 0):
        divs.append(div)

batch_size = divs[-1]
print(batch_size)

17


# CNN

In [16]:
def create_batches(input_array, label_array, batch_size):
    batched = []
    label_batched = []
    
    for i in range(np.floor(len(input_array)/batch_size).astype(int)):
        batched.append(np.expand_dims((np.array(input_array[i*batch_size:i*batch_size+batch_size])).astype(np.single),axis=1))
        label_batched.append(label_array[i*batch_size:i*batch_size+batch_size])
        
    return np.array(batched), label_batched

In [17]:
class Net(nn.Module):
    def __init__(self, h_layers):
        self.num_layers = len(h_layers)
        self.h_layers = h_layers      
        self.convs = []
        
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 256, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Train CNN for hyper-parameter testing

In [19]:
training_dataset_batch, training_labels_batch = create_batches(xTrainingSet, yTrainingSet, batch_size)
validation_dataset_batch, validation_labels_batch = create_batches(xValidationSet, yValidationSet, batch_size)

# model hyper-parameters
h_layers = ([batch_size, 64])
max_iters = 30
learning_rate = 0.001
momentum = 0.9
dont_run = True
# train neural network

if not dont_run:
    net = Net(h_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

    for epoch in range(max_iters):
        running_loss = 0.0

        for i, data in enumerate(training_dataset_batch):
            inputs = torch.from_numpy(data)
            labels = training_labels_batch[i] 
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, torch.from_numpy(np.array(labels).astype(np.longlong)))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 20 == 19:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 20))
                running_loss = 0.0

    # test validation set on model
    correct = 0
    total = 0

    with torch.no_grad():
        for i, data in enumerate(validation_dataset_batch):
            images = torch.from_numpy(data)
            labels = validation_labels_batch[i]
            labels = torch.from_numpy(np.array(labels).astype(np.longlong))
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            if (int(predicted[0]) != int(labels[0])):
                None

    print(f"validation set accuracy ({total} samples): {(100 * correct / total)}")

# Train CNN to predict unseen dataset

In [20]:
full_dataset = xTrainingSet + xValidationSet
full_labels = yTrainingSet + yValidationSet

full_dataset_batch, full_labels_batch = create_batches(full_dataset, full_labels, batch_size)

net = Net(h_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

for epoch in range(max_iters):
    running_loss = 0.0
    
    for i, data in enumerate(full_dataset_batch):
        inputs = torch.from_numpy(data)
        labels = full_labels_batch[i] 
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, torch.from_numpy(np.array(labels).astype(np.longlong)))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

[1,    20] loss: 2.597
[1,    40] loss: 2.160
[1,    60] loss: 1.877
[1,    80] loss: 1.464
[1,   100] loss: 1.053
[1,   120] loss: 0.822
[1,   140] loss: 0.768
[1,   160] loss: 0.721
[1,   180] loss: 0.615
[1,   200] loss: 0.385
[1,   220] loss: 0.324
[1,   240] loss: 0.418
[1,   260] loss: 0.416
[1,   280] loss: 0.377
[1,   300] loss: 0.400
[1,   320] loss: 0.287
[1,   340] loss: 0.363
[1,   360] loss: 0.349
[1,   380] loss: 0.287
[1,   400] loss: 0.346
[1,   420] loss: 0.264
[1,   440] loss: 0.197
[1,   460] loss: 0.305
[1,   480] loss: 0.296
[1,   500] loss: 0.255
[1,   520] loss: 0.180
[1,   540] loss: 0.220
[1,   560] loss: 0.240
[1,   580] loss: 0.220
[1,   600] loss: 0.186
[1,   620] loss: 0.301
[1,   640] loss: 0.187
[1,   660] loss: 0.226
[1,   680] loss: 0.203
[1,   700] loss: 0.272
[1,   720] loss: 0.237
[1,   740] loss: 0.128
[1,   760] loss: 0.207
[1,   780] loss: 0.144
[1,   800] loss: 0.204
[1,   820] loss: 0.175
[1,   840] loss: 0.148
[1,   860] loss: 0.209
[1,   880] 

[1,  7160] loss: 0.081
[1,  7180] loss: 0.057
[1,  7200] loss: 0.054
[1,  7220] loss: 0.116
[1,  7240] loss: 0.134
[1,  7260] loss: 0.045
[1,  7280] loss: 0.077
[1,  7300] loss: 0.071
[1,  7320] loss: 0.033
[1,  7340] loss: 0.060
[1,  7360] loss: 0.047
[1,  7380] loss: 0.038
[1,  7400] loss: 0.110
[1,  7420] loss: 0.031
[1,  7440] loss: 0.054
[1,  7460] loss: 0.067
[1,  7480] loss: 0.078
[1,  7500] loss: 0.106
[1,  7520] loss: 0.094
[1,  7540] loss: 0.095
[1,  7560] loss: 0.065
[1,  7580] loss: 0.048
[1,  7600] loss: 0.087
[1,  7620] loss: 0.047
[1,  7640] loss: 0.080
[1,  7660] loss: 0.048
[1,  7680] loss: 0.041
[1,  7700] loss: 0.052
[1,  7720] loss: 0.048
[1,  7740] loss: 0.032
[1,  7760] loss: 0.014
[1,  7780] loss: 0.091
[1,  7800] loss: 0.045
[1,  7820] loss: 0.045
[1,  7840] loss: 0.031
[1,  7860] loss: 0.036
[1,  7880] loss: 0.077
[1,  7900] loss: 0.059
[1,  7920] loss: 0.020
[1,  7940] loss: 0.081
[1,  7960] loss: 0.046
[1,  7980] loss: 0.055
[1,  8000] loss: 0.084
[1,  8020] 

[2,  4440] loss: 0.029
[2,  4460] loss: 0.027
[2,  4480] loss: 0.024
[2,  4500] loss: 0.008
[2,  4520] loss: 0.011
[2,  4540] loss: 0.037
[2,  4560] loss: 0.028
[2,  4580] loss: 0.032
[2,  4600] loss: 0.071
[2,  4620] loss: 0.054
[2,  4640] loss: 0.037
[2,  4660] loss: 0.012
[2,  4680] loss: 0.027
[2,  4700] loss: 0.019
[2,  4720] loss: 0.054
[2,  4740] loss: 0.068
[2,  4760] loss: 0.042
[2,  4780] loss: 0.026
[2,  4800] loss: 0.079
[2,  4820] loss: 0.038
[2,  4840] loss: 0.043
[2,  4860] loss: 0.023
[2,  4880] loss: 0.029
[2,  4900] loss: 0.018
[2,  4920] loss: 0.022
[2,  4940] loss: 0.053
[2,  4960] loss: 0.049
[2,  4980] loss: 0.038
[2,  5000] loss: 0.076
[2,  5020] loss: 0.061
[2,  5040] loss: 0.057
[2,  5060] loss: 0.065
[2,  5080] loss: 0.057
[2,  5100] loss: 0.038
[2,  5120] loss: 0.043
[2,  5140] loss: 0.019
[2,  5160] loss: 0.032
[2,  5180] loss: 0.110
[2,  5200] loss: 0.017
[2,  5220] loss: 0.024
[2,  5240] loss: 0.017
[2,  5260] loss: 0.026
[2,  5280] loss: 0.015
[2,  5300] 

[3,  1720] loss: 0.019
[3,  1740] loss: 0.023
[3,  1760] loss: 0.065
[3,  1780] loss: 0.034
[3,  1800] loss: 0.017
[3,  1820] loss: 0.029
[3,  1840] loss: 0.035
[3,  1860] loss: 0.017
[3,  1880] loss: 0.005
[3,  1900] loss: 0.016
[3,  1920] loss: 0.021
[3,  1940] loss: 0.026
[3,  1960] loss: 0.017
[3,  1980] loss: 0.029
[3,  2000] loss: 0.038
[3,  2020] loss: 0.025
[3,  2040] loss: 0.121
[3,  2060] loss: 0.050
[3,  2080] loss: 0.020
[3,  2100] loss: 0.017
[3,  2120] loss: 0.028
[3,  2140] loss: 0.047
[3,  2160] loss: 0.026
[3,  2180] loss: 0.022
[3,  2200] loss: 0.020
[3,  2220] loss: 0.040
[3,  2240] loss: 0.027
[3,  2260] loss: 0.012
[3,  2280] loss: 0.011
[3,  2300] loss: 0.012
[3,  2320] loss: 0.028
[3,  2340] loss: 0.045
[3,  2360] loss: 0.009
[3,  2380] loss: 0.025
[3,  2400] loss: 0.021
[3,  2420] loss: 0.029
[3,  2440] loss: 0.020
[3,  2460] loss: 0.013
[3,  2480] loss: 0.039
[3,  2500] loss: 0.053
[3,  2520] loss: 0.039
[3,  2540] loss: 0.011
[3,  2560] loss: 0.018
[3,  2580] 

[3,  8860] loss: 0.018
[3,  8880] loss: 0.010
[3,  8900] loss: 0.010
[3,  8920] loss: 0.012
[3,  8940] loss: 0.024
[3,  8960] loss: 0.026
[3,  8980] loss: 0.014
[3,  9000] loss: 0.007
[3,  9020] loss: 0.018
[3,  9040] loss: 0.018
[3,  9060] loss: 0.022
[3,  9080] loss: 0.029
[3,  9100] loss: 0.019
[3,  9120] loss: 0.019
[3,  9140] loss: 0.029
[3,  9160] loss: 0.011
[3,  9180] loss: 0.053
[3,  9200] loss: 0.020
[3,  9220] loss: 0.007
[3,  9240] loss: 0.025
[3,  9260] loss: 0.028
[3,  9280] loss: 0.028
[3,  9300] loss: 0.020
[3,  9320] loss: 0.031
[3,  9340] loss: 0.015
[3,  9360] loss: 0.029
[3,  9380] loss: 0.010
[3,  9400] loss: 0.019
[3,  9420] loss: 0.015
[3,  9440] loss: 0.005
[3,  9460] loss: 0.006
[3,  9480] loss: 0.007
[3,  9500] loss: 0.004
[3,  9520] loss: 0.035
[3,  9540] loss: 0.010
[3,  9560] loss: 0.003
[3,  9580] loss: 0.024
[3,  9600] loss: 0.015
[3,  9620] loss: 0.009
[3,  9640] loss: 0.016
[3,  9660] loss: 0.015
[3,  9680] loss: 0.011
[3,  9700] loss: 0.008
[3,  9720] 

...

[27,  7300] loss: 0.000
[27,  7320] loss: 0.000
[27,  7340] loss: 0.002
[27,  7360] loss: 0.000
[27,  7380] loss: 0.000
[27,  7400] loss: 0.002
[27,  7420] loss: 0.001
[27,  7440] loss: 0.000
[27,  7460] loss: 0.000
[27,  7480] loss: 0.001
[27,  7500] loss: 0.011
[27,  7520] loss: 0.000
[27,  7540] loss: 0.000
[27,  7560] loss: 0.000
[27,  7580] loss: 0.000
[27,  7600] loss: 0.003
[27,  7620] loss: 0.000
[27,  7640] loss: 0.001
[27,  7660] loss: 0.000
[27,  7680] loss: 0.001
[27,  7700] loss: 0.000
[27,  7720] loss: 0.000
[27,  7740] loss: 0.000
[27,  7760] loss: 0.000
[27,  7780] loss: 0.000
[27,  7800] loss: 0.000
[27,  7820] loss: 0.000
[27,  7840] loss: 0.000
[27,  7860] loss: 0.000
[27,  7880] loss: 0.000
[27,  7900] loss: 0.004
[27,  7920] loss: 0.002
[27,  7940] loss: 0.000
[27,  7960] loss: 0.000
[27,  7980] loss: 0.000
[27,  8000] loss: 0.000
[27,  8020] loss: 0.001
[27,  8040] loss: 0.000
[27,  8060] loss: 0.000
[27,  8080] loss: 0.000
[27,  8100] loss: 0.000
[27,  8120] loss

[28,  4280] loss: 0.000
[28,  4300] loss: 0.004
[28,  4320] loss: 0.000
[28,  4340] loss: 0.015
[28,  4360] loss: 0.001
[28,  4380] loss: 0.001
[28,  4400] loss: 0.000
[28,  4420] loss: 0.000
[28,  4440] loss: 0.000
[28,  4460] loss: 0.000
[28,  4480] loss: 0.001
[28,  4500] loss: 0.000
[28,  4520] loss: 0.001
[28,  4540] loss: 0.000
[28,  4560] loss: 0.008
[28,  4580] loss: 0.001
[28,  4600] loss: 0.007
[28,  4620] loss: 0.004
[28,  4640] loss: 0.002
[28,  4660] loss: 0.000
[28,  4680] loss: 0.001
[28,  4700] loss: 0.000
[28,  4720] loss: 0.000
[28,  4740] loss: 0.014
[28,  4760] loss: 0.010
[28,  4780] loss: 0.001
[28,  4800] loss: 0.000
[28,  4820] loss: 0.000
[28,  4840] loss: 0.023
[28,  4860] loss: 0.004
[28,  4880] loss: 0.000
[28,  4900] loss: 0.000
[28,  4920] loss: 0.000
[28,  4940] loss: 0.005
[28,  4960] loss: 0.000
[28,  4980] loss: 0.003
[28,  5000] loss: 0.012
[28,  5020] loss: 0.001
[28,  5040] loss: 0.007
[28,  5060] loss: 0.001
[28,  5080] loss: 0.001
[28,  5100] loss

[29,  1260] loss: 0.007
[29,  1280] loss: 0.000
[29,  1300] loss: 0.000
[29,  1320] loss: 0.003
[29,  1340] loss: 0.000
[29,  1360] loss: 0.000
[29,  1380] loss: 0.001
[29,  1400] loss: 0.000
[29,  1420] loss: 0.000
[29,  1440] loss: 0.000
[29,  1460] loss: 0.000
[29,  1480] loss: 0.000
[29,  1500] loss: 0.000
[29,  1520] loss: 0.002
[29,  1540] loss: 0.000
[29,  1560] loss: 0.000
[29,  1580] loss: 0.000
[29,  1600] loss: 0.000
[29,  1620] loss: 0.000
[29,  1640] loss: 0.000
[29,  1660] loss: 0.000
[29,  1680] loss: 0.000
[29,  1700] loss: 0.000
[29,  1720] loss: 0.000
[29,  1740] loss: 0.001
[29,  1760] loss: 0.001
[29,  1780] loss: 0.000
[29,  1800] loss: 0.000
[29,  1820] loss: 0.000
[29,  1840] loss: 0.000
[29,  1860] loss: 0.000
[29,  1880] loss: 0.000
[29,  1900] loss: 0.000
[29,  1920] loss: 0.000
[29,  1940] loss: 0.000
[29,  1960] loss: 0.000
[29,  1980] loss: 0.000
[29,  2000] loss: 0.000
[29,  2020] loss: 0.000
[29,  2040] loss: 0.000
[29,  2060] loss: 0.000
[29,  2080] loss

[29,  8100] loss: 0.000
[29,  8120] loss: 0.000
[29,  8140] loss: 0.004
[29,  8160] loss: 0.002
[29,  8180] loss: 0.001
[29,  8200] loss: 0.001
[29,  8220] loss: 0.000
[29,  8240] loss: 0.010
[29,  8260] loss: 0.000
[29,  8280] loss: 0.000
[29,  8300] loss: 0.000
[29,  8320] loss: 0.001
[29,  8340] loss: 0.004
[29,  8360] loss: 0.001
[29,  8380] loss: 0.001
[29,  8400] loss: 0.000
[29,  8420] loss: 0.000
[29,  8440] loss: 0.003
[29,  8460] loss: 0.001
[29,  8480] loss: 0.000
[29,  8500] loss: 0.000
[29,  8520] loss: 0.001
[29,  8540] loss: 0.000
[29,  8560] loss: 0.000
[29,  8580] loss: 0.001
[29,  8600] loss: 0.000
[29,  8620] loss: 0.000
[29,  8640] loss: 0.000
[29,  8660] loss: 0.000
[29,  8680] loss: 0.000
[29,  8700] loss: 0.000
[29,  8720] loss: 0.000
[29,  8740] loss: 0.000
[29,  8760] loss: 0.000
[29,  8780] loss: 0.000
[29,  8800] loss: 0.000
[29,  8820] loss: 0.001
[29,  8840] loss: 0.000
[29,  8860] loss: 0.000
[29,  8880] loss: 0.000
[29,  8900] loss: 0.000
[29,  8920] loss

[30,  5080] loss: 0.001
[30,  5100] loss: 0.000
[30,  5120] loss: 0.000
[30,  5140] loss: 0.000
[30,  5160] loss: 0.000
[30,  5180] loss: 0.000
[30,  5200] loss: 0.004
[30,  5220] loss: 0.000
[30,  5240] loss: 0.001
[30,  5260] loss: 0.000
[30,  5280] loss: 0.000
[30,  5300] loss: 0.003
[30,  5320] loss: 0.000
[30,  5340] loss: 0.000
[30,  5360] loss: 0.001
[30,  5380] loss: 0.000
[30,  5400] loss: 0.000
[30,  5420] loss: 0.000
[30,  5440] loss: 0.000
[30,  5460] loss: 0.000
[30,  5480] loss: 0.000
[30,  5500] loss: 0.000
[30,  5520] loss: 0.000
[30,  5540] loss: 0.000
[30,  5560] loss: 0.000
[30,  5580] loss: 0.000
[30,  5600] loss: 0.000
[30,  5620] loss: 0.000
[30,  5640] loss: 0.000
[30,  5660] loss: 0.000
[30,  5680] loss: 0.000
[30,  5700] loss: 0.000
[30,  5720] loss: 0.000
[30,  5740] loss: 0.000
[30,  5760] loss: 0.000
[30,  5780] loss: 0.000
[30,  5800] loss: 0.000
[30,  5820] loss: 0.000
[30,  5840] loss: 0.002
[30,  5860] loss: 0.000
[30,  5880] loss: 0.005
[30,  5900] loss

# Predict unseen dataset

In [22]:
separated_digits = []

for i, sample in enumerate(test_dataset):
    digits = []
    
    digits, n_digits = thresh_callback(sample)
        
    separated_digits.append(digits)
    
results = []

for i in range(len(test_dataset)):
    results.append([])

row = 0
col = 0
    
for i in range(int(num_test_digits / batch_size)):
    batch = []
    source = []
    count = 0
    
    while count < batch_size:
        batch.append(separated_digits[row][col])
        source.append((row, col))
        
        if(col < len(separated_digits[row]) - 1):
            col += 1
        else:
            row += 1
            col = 0
        
        count += 1
        
    batch = np.expand_dims(np.asarray(batch).astype(np.single), axis=1) 
    batch = torch.from_numpy(batch)
    
    output = net(batch)
    _, predicted = torch.max(output.data, 1)
    
    for j, pred in enumerate(predicted):
        coord = source[j]
        results[coord[0]].append(pred.item())
        
for r in results:
    pad = 5 - len(r)
    
    for i in range(pad):
        r.append(10)

In [ ]:
for i, sample in enumerate(test_dataset):
    displayGreyWindows(sample, "")
    print(results[i])

## Send data to csv

In [23]:
#print(str(results[0]))

Id = []
label = []
for i, x in enumerate(results):
    Id.append(i)
    string = ''.join([str(elem) for elem in x])
    label.append(string)
"""
print(Id[0],label[0])
print(df["Label"])
"""
data={"Label":label}

df = pd.DataFrame(data)

df.to_csv('sample.csv', index_label = "Id")
df.head()



,Label
0,910101010
1,017310
2,14083
3,010101010
4,467810
